In [1]:
import random
import sqlite3

#------------------database part-----------------
def create_conn(path):
    conn = None
    try:
        conn = sqlite3.connect(path)
        print("Connection to SQLite database is successful")
    except sqlite3.Error as e:
        print(f"Encountered error: '{e}'")

    return conn

def create_player(conn,player_name):
    cursor = conn.cursor()
    print("Please wait for a while...")
    create_player = f"""
    INSERT INTO
      players (name)
    VALUES
      ('{player_name}');
    """
    
    try:
        cursor.execute(create_player)
        conn.commit()
        print(f"Successfully created a player named {player_name}")
        return False
    except sqlite3.Error as e:
        print(f"Encountered error: '{e}'")
        print("Name already taken, please choose another name")
        return True
    
def get_player(conn,player_name):
    cursor = conn.cursor()
    result = None
    
    player_name = (player_name,)
    try:
        cursor.execute("""
        SELECT
          id,
          name
        FROM
          players
        WHERE
          name =?
        """,player_name)
        result = cursor.fetchone()
    except sqlite3.Error as e:
        print("The player does not exist.")
        print("Please recheck your spelling or capitalization.")
    
    return result

def save_marks(conn,player_name,mark):
    cursor = conn.cursor()
    player_id = get_player(conn,player_name)[0]
    print("Saving Score...")
    save_mark = f"""
    INSERT INTO
      marks (mark,player_id)
    VALUES
      ({mark},{player_id});
    """
    
    try:
        cursor.execute(save_mark)
        conn.commit()
        print(f"Successfully storing mark of {mark}")
    except sqlite3.Error as e:
        print(f"Encountered error: '{e}'")
    
def get_all_players_and_marks(conn):
    cursor = conn.cursor()
    result = None
    select_all_players_and_marks = """
    SELECT
      players.name,
      marks.mark
    FROM
      marks
      INNER JOIN players ON players.id = marks.player_id
    """
    try:
        cursor.execute(select_all_players_and_marks)
        result = cursor.fetchall()
        return result
    except sqlite3.Error as e:
        print(f"The error '{e}' occurred")

#------------------database part-----------------

def main_menu():
    start_game = 'h'
    print("---Welcome to Rock, Paper, Scissors---")
    print("-N- New Player")
    print("-E- Existing Player")
    while start_game[:1] not in ['n','e']:
        start_game = input("\nChoose between [N or E]: ").lower().strip()
    return start_game

def player_rps():
    while True:
        #validate input
        user_rps = input("Play Rock, Paper, Scissors. Please choose one among R or P or S: ").upper()
        if(user_rps[:1] not in ['R','P','S']):
            print("Please provide correct input. Please choose one among R or P or S: ")
            continue
        break
    
    return user_rps

def get_score(mark, prev_result, streaks):
    if(user_rps == generated_rps):
        print("\nDraw!!")
        return (mark, 'D', 0)
    elif((user_rps == 'P' and generated_rps == 'R') or (user_rps == 'R' and generated_rps == 'S') 
        or (user_rps == 'S' and generated_rps == 'P')):
        print("\nYou Win!!")
        if(prev_result == 'W'):
            return ((mark * (streaks + 1)),'W', (streaks + 1))
        else:
            return ((mark + 2), 'W', 1)
    else:
        print("\nCode Win!!")
        if(prev_result == 'L'):
            return (mark - ((2*(streaks + 1)-1)),'L', streaks +1)
        else:
            return ((mark - 1), 'L', 1)
    
def confirm_proceed():
    proceed_input = 'k' #initialize variable value
    while proceed_input[:1] not in ['y','n']:
        proceed_input = input("\nDo you have any player left to play? 'Y' - Yes or 'N' - No: ").lower().strip()
    
    return proceed_input[:1]

def output_scores():
    players_and_marks = get_all_players_and_marks(conn)
    players_and_marks = sorted(players_and_marks, key = lambda score: score[1],reverse=True)

    print(("\nPLAYER\t|SCORES").expandtabs(10))
    print("-----------------")
    for i in range(len(players_and_marks)):
        print((f'{players_and_marks[i][0]}\t|{players_and_marks[i][1]}').expandtabs(10))

conn = create_conn("database/mark_db.sqlite")

NUM_OF_ROUNDS = 5
proceed = 'y'
while proceed == 'y':
    ask_name = True
    start_game = main_menu()
    while(ask_name):
        print("Type 'r' to return")
        player_name = input("Name: ") #ask for player name
        if(player_name == 'r'):
            start_game = main_menu()
            continue
        if(start_game == 'n'):
            ask_name = create_player(conn,player_name)
        elif(start_game == 'e'):
            if (get_player(conn,player_name)):
                ask_name = False
            else:
                print("The player does not exist.")
                print("Please recheck your spelling or capitalization.")
    rps_mark = 0
    prev_result = 'N'
    streaks = 0
    for i in range(NUM_OF_ROUNDS):
        generated_rps = random.choice(['R', 'P', 'S'])
        #ask user input
        user_rps = player_rps()
        
        #output user choice and generated value
        print("\nYou choose: " + user_rps)
        print("Code generate: " + generated_rps)

        #do comparison to see which side win and add marks
        rps_mark,prev_result, streaks = get_score(rps_mark, prev_result, streaks)
        print(rps_mark)

    save_marks(conn,player_name,rps_mark)
    output_scores()
    proceed = confirm_proceed()
    
conn.close()

Connection to SQLite database is successful
---Welcome to Rock, Paper, Scissors---
-N- New Player
-E- Existing Player

Choose between [N or E]: n
Type 'r' to return
Name: Hazqeel
Please wait for a while...
Successfully created a player named Hazqeel
Play Rock, Paper, Scissors. Please choose one among R or P or S: r

You choose: R
Code generate: S

You Win!!
2
Play Rock, Paper, Scissors. Please choose one among R or P or S: p

You choose: P
Code generate: S

Code Win!!
1
Play Rock, Paper, Scissors. Please choose one among R or P or S: p

You choose: P
Code generate: R

You Win!!
3
Play Rock, Paper, Scissors. Please choose one among R or P or S: s

You choose: S
Code generate: R

Code Win!!
2
Play Rock, Paper, Scissors. Please choose one among R or P or S: s

You choose: S
Code generate: P

You Win!!
4
Saving Score...
Successfully storing mark of 4

PLAYER    |SCORES
-----------------
Abu       |5
Chun Keat |5
Mogana    |5
Hazqeel   |4

Do you have any player left to play? 'Y' - Yes or '